In [3]:
library(dplyr)

In [4]:
meta_data=read.csv("/home//workspace/IHA_Figure_Revision/Dataset/scRNA_meta_data-2024-05-09.csv")
meta_data$sample.drawDate <- as.Date(meta_data$sample.drawDate)
 
meta_data <- meta_data %>%
  mutate(Flu_Year = case_when(
    sample.drawDate <= as.Date('2020-07-01') ~ '2019-2020',
    sample.drawDate > as.Date('2020-07-01') & sample.drawDate <= as.Date('2021-07-01') ~ '2020-2021',
    sample.drawDate > as.Date('2021-07-01') & sample.drawDate <= as.Date('2022-07-01') ~ '2021-2022',
    sample.drawDate > as.Date('2022-07-01') & sample.drawDate <= as.Date('2023-07-01') ~ '2022-2023',
    TRUE ~ "NA"
  ))

meta_data <- meta_data %>%
  mutate(Flu_Day = case_when(
    sample.visitName	%in% c("Flu Year 1 Day 0","Flu Year 2 Day 0","Immune Variation Day 0") ~ 'Day_0',
    sample.visitName	%in% c("Flu Year 1 Day 7","Flu Year 2 Day 7",'Immune Variation Day 7') ~ 'Day_7',
    sample.visitName	%in% c("Flu Year 1 Day 90","Flu Year 2 Day 90",'Immune Variation Day 90') ~ 'Day_90',
    sample.visitName	%in% c("Flu Year 1 Stand-Alone","Flu Year 2 Stand-Alone","Flu Year 3 Stand-Alone") ~ 'Stand-Alone',
    TRUE ~ sample.visitName
  ))

meta_data <- meta_data %>%
  mutate(Visit_Type = case_when(
    startsWith(sample.visitName, "Flu Year") ~ "FluYear",
    startsWith(sample.visitName, "Immune Variation") ~ "ImmVar",
    TRUE ~ sample.visitName  
  ))

meta_data <- meta_data %>%
  mutate(Year_N = case_when(
    startsWith(sample.visitName, "Flu Year 1") ~ "Year 1",
    startsWith(sample.visitName, "Flu Year 2") ~ "Year 2",
      startsWith(sample.visitName, "Flu Year 2") ~ "Year 3",
    startsWith(sample.visitName, "Immune Variation") ~ "ImmVar",
    TRUE ~ sample.visitName  
  ))

meta_data <- meta_data %>% 
  dplyr::group_by(Year_N, subject.subjectGuid, Visit_Type) %>% 
  dplyr::arrange(Flu_Day) %>% 
  dplyr::mutate(Flu_Year = first(Flu_Year))

meta_data$Visit_Flu_Year<-paste0(meta_data$Visit_Type,"_",meta_data$Flu_Year,"_",meta_data$Flu_Day)

In [6]:
write.csv(meta_data,"meta_data.csv")